In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive

/content/drive


In [ ]:
cd MyDrive

/content/drive/MyDrive


In [ ]:
cd cdQA-bert_qa_vGPU

/content/drive/MyDrive/cdQA-bert_qa_vGPU


In [ ]:
pip install -e .

Obtaining file:///content/drive/MyDrive/cdQA-bert_qa_vGPU
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 114.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 12.8 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=62ff75ebbb9cd3e1d95e6a2598e0614b14dff8c73bc718fe0fe7c966572ded16
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
  Attempting uninstall: pyjwt
    Found existing installation: PyJWT 2.3.0
    Uninstalling 

In [ ]:
pip install cdqa

# reader Bert training

In [ ]:
import os
import torch
import joblib
from cdqa.reader.bertqa_sklearn import BertProcessor, BertQA

/content/drive/MyDrive/cdQA-bert_qa_vGPU/cdqa/reader/bertqa_sklearn.py:36: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# pre-process examples
train_processor = BertProcessor(do_lower_case=True, is_training=True)
train_examples, train_features = train_processor.fit_transform(X='data/KorQuAD_v1.0_train.json') # korean.csv는 korquad 데이터로 학습시켜야함

# train the model
# train_batch_size를 4보다 크게 할 경우, colab에서 제공하는 GPU로는 메모리가 부족해서 에러가 발생
reader = BertQA(train_batch_size=2,
               learning_rate=3e-5,
               num_train_epochs=2,
               do_lower_case=True,
               fp16=False,
               output_dir='models')

reader.fit(X=(train_examples, train_features))

100%|██████████| 407873900/407873900 [00:21<00:00, 18610702.00B/s]


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/130051 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_pretrained_bert/optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1519.)
  next_m.mul_(beta1).add_(1 - beta1, grad)


In [ ]:
# send it to GPU
reader.model.to('cuda')
reader.device = torch.device('cuda')

# save it locally
joblib.dump(reader, os.path.join(reader.output_dir, 'bert_qa_korquad_vGPU.joblib'))